## **Imports**

In [272]:
import pandas as pd
import numpy as np
import importlib
from application.modeling import disbursements
from application.modeling import interest_income
from application.modeling import other_income
from application.modeling import helper
from application.modeling import depreciation
from application.modeling import expenses
from application.modeling import income_statement
from application.modeling import direct_cashflow
from application.modeling import balance_sheet
from application.modeling import statement_of_cashflows
from application.modeling import loan_book

pd.options.display.max_columns = None
pd.options.display.max_rows = 100
pd.options.display.float_format = "{:,.2f}".format

## **Inputs**

In [273]:
VALUATION_DATE = "2023-01"
MONTHS_TO_FORECAST = 12
IMTT = 0.01

In [274]:
parameters = pd.read_csv("./data_templates/parameters.csv", index_col=0)
parameters.columns = pd.period_range(VALUATION_DATE,periods=int(parameters.columns[-1]), freq="M")
parameters.head()

,2023-01,2023-02,2023-03,2023-04,2023-05,2023-06,2023-07,2023-08,2023-09,2023-10,2023-11,2023-12
WORKING_CAPITAL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
NEW_TRADE_PAYABLES,500.00,500.00,500.00,500.00,500.00,500.00,500.00,500.00,500.00,500.00,500.00,500.00
NEW_TRADE_RECEIVABLES,500.00,500.00,500.00,500.00,500.00,500.00,500.00,500.00,500.00,500.00,500.00,500.00
RECEIPTS_FROM_TRADE_RECEIVABLES,"1,000.00","1,000.00","1,000.00","1,000.00","1,000.00","1,000.00","1,000.00","1,000.00","1,000.00","1,000.00","1,000.00","1,000.00"
PAYMENTS_TO_TRADE_PAYABLES,"1,000.00","1,000.00","1,000.00","1,000.00","1,000.00","1,000.00","1,000.00","1,000.00","1,000.00","1,000.00","1,000.00","1,000.00"


In [275]:
details_of_existing_assets = pd.read_csv("./data_templates/details_of_existing_assets.csv")
details_of_existing_assets

,asset,depreciation,net_book_value,remaining_useful_life,cost,useful_life_when_purchased,method
0,Vehicles,0.20,10000,5,20000,10,straight_line
1,Buildings,0.10,50000,20,80000,25,straight_line
2,Computers,0.20,10000,5,20000,10,straight_line
3,Furniture,0.10,50000,20,80000,25,straight_line


In [276]:
details_of_new_assets = pd.read_csv("./data_templates/details_of_new_assets.csv")
details_of_new_assets

,asset,depreciation,net_book_value,remaining_useful_life,method,purchase_date,cost
0,Vehicles,0.25,10000,5,straight_line,01/06/2023,10000
1,Buildings,0.10,50000,20,straight_line,01/09/2023,50000


In [277]:
details_of_existing_long_term_borrowing = pd.read_csv("./data_templates/details_of_existing_long_term_borrowing.csv")
details_of_existing_long_term_borrowing

,company,principal,interest_rate,remaining_loan_term,loan_term,method
0,Zimnat Lion Insurance,92800,0.06,60,60,straight_line
1,Zimnat Lion Insurance,50000,0.06,12,12,straight_line
2,Grandre,160000,0.06,36,36,straight_line
3,Minerva,15000,0.08,36,36,straight_line
4,Masawara,5613,0.06,60,60,straight_line
5,Zam-Cargo,364000,0.05,36,36,straight_line


In [278]:
details_of_new_long_term_borrowing = pd.read_csv("./data_templates/details_of_new_long_term_borrowing.csv")
details_of_new_long_term_borrowing

,company,principal,interest_rate,loan_term,remaining_loan_term,loan_start_date,method
0,ZMF,200000,0.34,60,60,01/07/2023,straight_line
1,ZAM,400000,0.25,60,60,01/07/2023,reducing_balance
2,NBS,500000,0.18,60,60,01/09/2023,straight_line
3,ZB,500000,0.21,60,60,01/10/2023,straight_line


In [279]:
details_of_new_short_term_borrowing = pd.read_csv("./data_templates/details_of_new_short_term_borrowing.csv")
details_of_new_short_term_borrowing

,company,principal,interest_rate,loan_term,remaining_loan_term,loan_start_date,method
0,ZMF,200000,0.34,12,12,01/07/2023,straight_line
1,ZAM,400000,0.25,12,12,01/07/2023,reducing_balance
2,NBS,500000,0.18,12,12,01/09/2023,straight_line
3,ZB,500000,0.21,12,12,01/10/2023,straight_line


In [280]:
details_of_existing_short_term_borrowing = pd.read_csv("./data_templates/details_of_existing_short_term_borrowing.csv")
details_of_existing_short_term_borrowing

,company,principal,interest_rate,remaining_loan_term,loan_term,method
0,Zimnat Life,50000,0.14,10,12,straight_line
1,Zimnat Life,30000,0.13,12,12,straight_line
2,Grandre,100000,0.12,7,12,straight_line
3,FCB,120383,0.18,5,12,straight_line
4,Nurture,500000,0.25,8,12,straight_line
5,NMB,450000,0.22,11,12,straight_line
6,Beramasamba,30000,0.13,3,12,straight_line
7,Dandemutande,25000,0.12,7,12,reducing_balance


In [281]:
static_inputs_income_statement = pd.read_csv("./data_templates/static_inputs_income_statement.csv", index_col=0)
static_inputs_income_statement

,Jan-23,Feb-23,Mar-23,Apr-23,May-23,Jun-23,Jul-23,Aug-23,Sep-23,Oct-23,Nov-23,Dec-23
CILL,13152,11585,13943,17555,13073,11021,13843,17989,19692,16873,15675,10161
Bonus Provision,14297,10995,17629,19467,11016,17869,16439,17892,16863,17916,18529,10878
Retrenchments,19268,14887,14859,16331,18571,18684,17208,15276,12062,10064,18006,12568
Staff Welfare,15463,12027,12695,19687,15258,15618,16736,10391,15892,13561,16184,13099
Donations,16278,18392,13104,17215,12454,18996,12731,18154,19762,15056,18110,13840
Legal Fees,0,0,0,0,0,0,0,0,0,0,0,0
Strategic Expenses,0,0,0,0,0,0,0,0,0,0,0,0
Rental Income,0,0,0,0,0,0,0,0,0,0,0,0
Interest Received,0,0,0,0,0,0,0,0,0,0,0,0
Dividends Received,0,0,0,0,0,0,0,0,0,0,0,0


In [282]:
variable_inputs_income_statement = pd.read_csv("./data_templates/variable_inputs_income_statement.csv", index_col=0)
variable_inputs_income_statement

,Aug-22,Sep-22,Oct-22,Nov-22,Dec-22,Jan-23
Training,6028,5502,9493,5870,9488,5206
Travel Costs,6484,5863,7790,5563,9191,6757
Entertainment,6678,8242,9636,6059,6722,8314
Marketing Costs,8157,7625,7729,6597,8060,7693
Group Marketing Costs,9891,8627,6363,6981,6663,6529
Rental Costs,7038,8302,7237,6306,9029,7675
Subscriptions,6282,5709,9744,9413,8748,9779
Insurance,5663,6998,8445,8743,6495,8304
Repairs And Maintenance,8763,6853,9659,6291,8581,8457
Utilities,6636,8696,7999,8152,5698,7160


In [283]:
opening_balances = pd.read_csv("./data_templates/opening_balances.csv")
opening_balances

,TRADE_RECEIVABLES,TRADE_PAYABLES,LOAN_BOOK,CASH_ON_HAND,ISSUED_SHARE_CAPITAL,SHARE_PREMIUM,OTHER_COMPONENTS_OF_EQUITY,TREASURY_SHARES,RETAINED_EARNINGS,LONG_TERM_LOANS,SHORT_TERM_LOANS,INTERCOMPANY_LOANS,DEFERED_TAXATION,OTHER_RECEIVABLES,OTHER_PAYABLES,PROVISION_FOR_TAXATION,PROVISION_FOR_CREDIT_LOSS,INVESTMENT_IN_SUBSIDIARIES,INVESTMENT_IN_ASSOCIATES,INVESTMENT_PROPERTIES,EQUITY_INVESTMENTS,LONG_TERM_MONEY_MARKET_INVESTMENTS,LOANS_TO_RELATED_ENTITIES,TREASURY_SHARES.1,INTANGIBLE_ASSETS,INVENTORIES,OTHER_RECEIVABLES.1,INTERGROUP_RECEIVABLES,SHORT_TERM_MONEY_MARKET_INVESTMENTS
0,1000000,940000,2800000,331000,7000,110,0,0,200000,200000,50000,0,0,0,0,0,10000,0,10000,0,20000,0,0,0,0,0,0,0,0


In [284]:
existing_loans = pd.read_csv("./data_templates/existing_loans.csv", index_col=0)
existing_loans

,disbursement_date,maturity_date,loan_amount,frequency,loan_term_months,remaining_term_months,interest_rate,repayment_amount,outstanding_balance,admin_fee_percentage,credit_insurance_fee_percentage
loan_number,,,,,,,,,,,
40b0775c,43536,46386,2400,12,94,48,0.00,31.39,"1,351.78",0.06,0.05
f5c36555,43873,46386,4600,12,83,48,0.01,92.11,"3,246.86",0.06,0.07
75656bce,44068,46386,2400,12,77,48,0.01,47.81,"1,739.68",0.08,0.05
a7222f00,44611,46021,3800,12,47,36,0.00,83.95,"2,935.72",0.09,0.08
ad00cbf5,43747,46751,5000,12,99,60,0.00,52.09,"3,066.98",0.07,0.05
321ec75b,44137,46751,1700,12,86,60,0.00,24.25,"1,258.85",0.08,0.10
6747e1d4,43606,46021,1600,12,80,36,0.01,31.43,911.36,0.05,0.10
5131c13c,44591,46386,1000,12,60,48,0.01,20.90,835.18,0.05,0.08
a7222f00_1,43520,46021,1200,12,83,36,0.01,18.50,595.83,0.06,0.05


## **New Disbursements**

In [285]:
new_disbursements_df  = disbursements.calculate_new_disbursements(parameters=parameters)
new_disbursements_df

,b2b_disbursements,sme_disbursements,consumer_ssb_disbursements,consumer_pvt_disbursements,total
2023-01,"10,000.00","360,000.00","48,000.00","60,000.00","490,000.00"
2023-02,"10,000.00","432,000.00","60,000.00","75,000.00","592,000.00"
2023-03,"10,000.00","480,000.00","72,000.00","108,000.00","706,000.00"
2023-04,"10,000.00","360,000.00","80,000.00","120,000.00","610,000.00"
2023-05,"10,000.00","324,000.00","96,000.00","144,000.00","622,000.00"
2023-06,"10,000.00","378,000.00","96,000.00","162,000.00","712,000.00"
2023-07,"10,000.00","420,000.00","112,000.00","189,000.00","808,000.00"
2023-08,"10,000.00","595,000.00","120,000.00","202,500.00","1,010,000.00"
2023-09,"10,000.00","700,000.00","120,000.00","225,000.00","1,160,000.00"
2023-10,"10,000.00","990,000.00","120,000.00","225,000.00","1,450,000.00"


### **Calculate Repayment Amount For New Disbursements**

In [286]:
monthly_repayment_new_disbursements_df = interest_income.calculate_monthly_repayments_new_disbursements(new_disbursements_df=new_disbursements_df, parameters=parameters)
monthly_repayment_new_disbursements_df

,sme_monthly_repayment,b2b_monthly_repayment,consumer_ssb_monthly_repayment,consumer_pvt_monthly_repayment,total
2023-01,"81,451.18","1,627.45","8,006.34","10,007.93","101,092.91"
2023-02,"97,741.42","1,627.45","10,007.93","12,509.91","121,886.71"
2023-03,"108,601.58","1,627.45","12,009.51","18,014.27","140,252.81"
2023-04,"81,451.18","1,627.45","11,172.05","16,758.08","111,008.77"
2023-05,"73,306.06","1,627.45","13,406.46","20,109.69","108,449.68"
2023-06,"85,523.74","1,627.45","13,406.46","22,623.41","123,181.06"
2023-07,"95,026.38","1,627.45","15,640.87","26,393.97","138,688.68"
2023-08,"134,620.70","1,627.45","14,887.07","28,279.26","179,414.48"
2023-09,"158,377.30","1,627.45","14,887.07","31,421.40","206,313.22"
2023-10,"223,990.75","1,627.45","13,705.47","31,421.40","270,745.08"


### **Generate Loan Schedules For New Disbursements**

In [287]:
loan_schedules_for_all_new_disbursements = (
    interest_income.generate_loan_schedules_for_all_new_disbursements(
        new_disbursements_df=new_disbursements_df,
        parameters=parameters,
        monthly_repayment_new_disbursements_df=monthly_repayment_new_disbursements_df,
        months_to_forecast=MONTHS_TO_FORECAST,
    )
)
loan_schedules_for_all_new_disbursements.keys()

dict_keys(['sme_loan_schedules', 'b2b_loan_schedules', 'consumer_ssb_loan_schedules', 'consumer_pvt_loan_schedules'])

### **Capital Repayment New Disbursement**

In [288]:
capital_repayment_new_disbursements_df = interest_income.generate_capital_repayment_new_disbursements_df(loan_schedules_for_all_new_disbursements=loan_schedules_for_all_new_disbursements)
capital_repayment_new_disbursements_df

,sme_capital_repayments,b2b_capital_repayments,consumer_pvt_capital_repayments,consumer_ssb_capital_repayments,total
Feb-2023,"47,251.18",627.45,"4,607.93","3,686.34","56,172.91"
Mar-2023,"108,441.46","1,317.65","10,782.55","8,626.04","129,167.71"
Apr-2023,"181,744.98","2,076.87","20,047.25","14,931.90","218,801.00"
May-2023,"246,261.93","2,912.01","27,809.58","20,247.82","297,231.35"
Jun-2023,"312,182.88","3,830.67","37,462.14","26,836.59","380,312.28"
Jul-2023,"391,453.99","4,841.19","48,877.14","34,018.35","479,190.67"
Aug-2023,"402,317.32","5,952.76","62,660.06","42,640.87","513,571.01"
Sep-2023,"420,891.75","7,175.49","78,353.72","50,565.62","556,986.58"
Oct-2023,"444,152.19","8,520.50","96,576.95","59,203.59","608,453.22"
Nov-2023,"534,836.21","10,000.00","106,432.35","59,431.04","710,699.61"


### **Interest Income New Disbursements**


In [289]:
interest_income_new_disbursements_df = interest_income.generate_interest_income_new_disbursements_df(
    loan_schedules_for_all_new_disbursements=loan_schedules_for_all_new_disbursements
)

interest_income_new_disbursements_df

,sme_interest_income,b2b_interest_income,consumer_pvt_interest_income,consumer_ssb_interest_income,total
Feb-2023,"34,200.00","1,000.00","5,400.00","4,320.00","44,920.00"
Mar-2023,"70,751.14","1,937.25","11,735.29","9,388.23","93,811.91"
Apr-2023,"106,049.20","2,805.49","20,484.86","15,091.89","144,431.43"
May-2023,"122,983.43","3,597.80","29,480.60","20,948.01","177,009.85"
Jun-2023,"130,368.54","4,306.60","39,937.74","27,765.71","202,378.60"
Jul-2023,"136,621.17","4,923.53","51,146.15","33,990.42","226,681.27"
Aug-2023,"139,333.04","5,439.41","63,757.21","41,008.77","249,538.43"
Sep-2023,"157,637.89","5,844.14","76,342.80","47,971.09","287,795.92"
Oct-2023,"184,153.18","6,126.59","89,540.97","54,220.18","334,040.92"
Nov-2023,"236,008.72","6,274.54","101,099.04","59,691.86","403,074.16"


### **Calculate Admin Fee On New Disbursements**

In [290]:
admin_fee_for_all_new_disbursements_df = other_income.calculate_admin_fee_for_all_new_disbursements(
    new_disbursements_df=new_disbursements_df,
    parameters=parameters,
    months_to_forecast=MONTHS_TO_FORECAST,
)

admin_fee_for_all_new_disbursements_df

,sme_admin_fee,b2b_admin_fee,consumer_ssb_admin_fee,consumer_pvt_admin_fee,total
Jan-2023,"3,000.00",10.00,184.62,272.73,"3,467.34"
Feb-2023,"6,600.00",20.00,415.38,613.64,"7,649.02"
Mar-2023,"10,600.00",30.00,692.31,"1,104.55","12,426.85"
Apr-2023,"13,600.00",40.00,"1,000.00","1,650.00","16,290.00"
May-2023,"16,300.00",50.00,"1,369.23","2,304.55","20,023.78"
Jun-2023,"19,450.00",60.00,"1,738.46","3,040.91","24,289.37"
Jul-2023,"19,950.00",70.00,"2,169.23","3,900.00","26,089.23"
Aug-2023,"21,308.33",80.00,"2,630.77","4,820.45","28,839.56"
Sep-2023,"23,141.67",90.00,"3,092.31","5,843.18","32,167.16"
Oct-2023,"28,391.67",100.00,"3,553.85","6,865.91","38,911.42"


### **Calculate Credit Life Insurance On New Disbursement**

In [291]:
credit_insurance_fee_for_all_new_disbursements = other_income.calculate_credit_insurance_fee_for_all_new_disbursements(
        new_disbursements_df=new_disbursements_df,
        parameters=parameters,
        months_to_forecast=MONTHS_TO_FORECAST,
    )

credit_insurance_fee_for_all_new_disbursements

,sme_credit_insurance_fee,b2b_credit_insurance_fee,consumer_ssb_credit_insurance_fee,consumer_pvt_credit_insurance_fee,total
Jan-2023,"1,200.00",0.00,73.85,109.09,"1,382.94"
Feb-2023,"2,640.00",0.00,166.15,245.45,"3,051.61"
Mar-2023,"4,240.00",0.00,276.92,441.82,"4,958.74"
Apr-2023,"5,440.00",0.00,400.00,660.00,"6,500.00"
May-2023,"6,520.00",0.00,547.69,921.82,"7,989.51"
Jun-2023,"7,780.00",0.00,695.38,"1,216.36","9,691.75"
Jul-2023,"7,980.00",0.00,867.69,"1,560.00","10,407.69"
Aug-2023,"8,523.33",0.00,"1,052.31","1,928.18","11,503.82"
Sep-2023,"9,256.67",0.00,"1,236.92","2,337.27","12,830.86"
Oct-2023,"11,356.67",0.00,"1,421.54","2,746.36","15,524.57"


### **Calculate For Credit Loss For New Disbursements**

In [292]:
provision_for_credit_loss_for_all_new_disbursements = (
    expenses.calculate_provision_for_credit_loss_for_all_new_disbursements(
        new_disbursements_df=new_disbursements_df, parameters=parameters
    )
)
provision_for_credit_loss_for_all_new_disbursements

,sme_provision_for_credit_loss,b2b_provision_for_credit_loss,consumer_ssb_provision_for_credit_loss,consumer_pvt_provision_for_credit_loss,total
Jan-2023,"7,200.00",100.00,960.00,"1,800.00","10,060.00"
Feb-2023,"8,640.00",100.00,"1,200.00","2,250.00","12,190.00"
Mar-2023,"9,600.00",100.00,"1,440.00","3,240.00","14,380.00"
Apr-2023,"7,200.00",100.00,"1,600.00","3,600.00","12,500.00"
May-2023,"6,480.00",100.00,"1,920.00","4,320.00","12,820.00"
Jun-2023,"7,560.00",100.00,"1,920.00","4,860.00","14,440.00"
Jul-2023,"8,400.00",100.00,"2,240.00","5,670.00","16,410.00"
Aug-2023,"11,900.00",100.00,"2,400.00","6,075.00","20,475.00"
Sep-2023,"14,000.00",100.00,"2,400.00","6,750.00","23,250.00"
Oct-2023,"19,800.00",100.00,"2,400.00","6,750.00","29,050.00"


## **Existing Loans**

### **Generate Loan Schedules For Existing Loans**


In [293]:
existing_loans_schedules = interest_income.generate_loan_schedules_existing_loans(
    outstanding_balance=existing_loans["outstanding_balance"],
    interest_rate_monthly=existing_loans["interest_rate"],
    repayment_amount_monthly=existing_loans["repayment_amount"],
    valuation_date=VALUATION_DATE, 
    months_to_project=MONTHS_TO_FORECAST,
)
existing_loans_schedules.keys()

dict_keys(['interest', 'capital_repayment', 'outstanding_at_start'])

### **Capital Repayment for Existing Loans**

In [294]:
capital_repayment_existing_loans = existing_loans_schedules['capital_repayment'].sum()
capital_repayment_existing_loans

Jan-2023   884.89
Feb-2023   891.70
Mar-2023   898.59
Apr-2023   905.55
May-2023   912.59
Jun-2023   919.70
Jul-2023   926.89
Aug-2023   934.16
Sep-2023   941.52
Oct-2023   948.95
Nov-2023   956.47
Dec-2023     0.00
dtype: float64

### **Intererest Income For Existing Loans**

In [295]:
interest_income_existing_loans = existing_loans_schedules['interest'].sum()
interest_income_existing_loans

Jan-2023   290.78
Feb-2023   283.97
Mar-2023   277.09
Apr-2023   270.13
May-2023   263.09
Jun-2023   255.98
Jul-2023   248.78
Aug-2023   241.51
Sep-2023   234.16
Oct-2023   226.73
Nov-2023   219.21
Dec-2023     0.00
dtype: float64

### **Calculate Other Income Existing Loans**

In [296]:
other_income_existing_loans_df = other_income.calculate_other_income_existing_loans(existing_loans=existing_loans, valuation_date=VALUATION_DATE, months_to_forecast=MONTHS_TO_FORECAST)
other_income_existing_loans_df

,credit_insurance_fee_existing_loans,admin_fee_existing_loans,total
Jan-2023,61.55,66.22,127.78
Feb-2023,61.55,66.22,127.78
Mar-2023,61.55,66.22,127.78
Apr-2023,61.55,66.22,127.78
May-2023,61.55,66.22,127.78
Jun-2023,61.55,66.22,127.78
Jul-2023,61.55,66.22,127.78
Aug-2023,61.55,66.22,127.78
Sep-2023,61.55,66.22,127.78
Oct-2023,61.55,66.22,127.78


## **Interest Income**

In [297]:
total_interest_income = interest_income.aggregate_new_and_existing_loans_interest_income(
    interest_income_new_disbursements_df=interest_income_new_disbursements_df,
    interest_income_existing_loans= interest_income_existing_loans,
    valuation_date= VALUATION_DATE, 
    months_to_forecast= MONTHS_TO_FORECAST
)
total_interest_income

Jan-2023       290.78
Feb-2023    45,203.97
Mar-2023    94,088.99
Apr-2023   144,701.56
May-2023   177,272.94
Jun-2023   202,634.57
Jul-2023   226,930.05
Aug-2023   249,779.94
Sep-2023   288,030.08
Oct-2023   334,267.64
Nov-2023   403,293.37
Dec-2023   475,262.01
dtype: float64

## **Other Income**

In [298]:
other_income_df = other_income.aggregate_other_income(
    admin_fee_for_all_new_disbursements_df=admin_fee_for_all_new_disbursements_df,
    admin_fee_existing_loans=other_income_existing_loans_df['admin_fee_existing_loans'],
    credit_insurance_fee_existing_loans=other_income_existing_loans_df['credit_insurance_fee_existing_loans'],
    credit_insurance_fee_for_all_new_disbursements_df=credit_insurance_fee_for_all_new_disbursements, 
    valuation_date=VALUATION_DATE, 
    months_to_forecast=MONTHS_TO_FORECAST
)
other_income_df

,admin_fee,credit_insurance_fee,total
Jan-2023,"7,000.91","2,827.43","9,828.34"
Feb-2023,"15,364.26","6,164.77","21,529.03"
Mar-2023,"24,919.93","9,979.04","34,898.96"
Apr-2023,"32,646.22","13,061.55","45,707.78"
May-2023,"40,113.77","16,040.57","56,154.35"
Jun-2023,"48,644.96","19,445.05","68,090.01"
Jul-2023,"52,244.68","20,876.94","73,121.62"
Aug-2023,"57,745.34","23,069.20","80,814.54"
Sep-2023,"64,400.53","25,723.28","90,123.81"
Oct-2023,"77,889.07","31,110.69","108,999.76"


## **Depreciation**

In [314]:
importlib.reload(depreciation)

<module 'application.modeling.depreciation' from 'c:\\Users\\HP\\Desktop\\Claxon\\Budgeting\\manage-tenants-template\\application\\modeling\\depreciation.py'>

In [315]:
depreciations_and_nbvs = depreciation.calculate_depreciations_and_nbvs(
    details_of_existing_assets=details_of_existing_assets,
    details_of_new_assets=details_of_new_assets,
    valuation_date=VALUATION_DATE,
    months_to_forecast=MONTHS_TO_FORECAST
)
depreciations_and_nbvs.keys()

dict_keys(['nbvs', 'dpns'])

In [316]:
depreciations_and_nbvs['nbvs']

,Buildings,Computers,Furniture,Vehicles,total
Jan-2023,"42,000.00","6,000.00","42,000.00","6,000.00","96,000.00"
Feb-2023,"34,000.00","2,000.00","34,000.00","2,000.00","72,000.00"
Mar-2023,"26,000.00","-2,000.00","26,000.00","-2,000.00","48,000.00"
Apr-2023,"18,000.00","-6,000.00","18,000.00","-6,000.00","24,000.00"
May-2023,"10,000.00","-10,000.00","10,000.00","-10,000.00",0.00
Jun-2023,"2,000.00",0.00,"2,000.00","7,500.00","11,500.00"
Jul-2023,"-6,000.00",0.00,"-6,000.00","5,000.00","-7,000.00"
Aug-2023,"-14,000.00",0.00,"-14,000.00","2,500.00","-25,500.00"
Sep-2023,"23,000.00",0.00,"-22,000.00",0.00,"1,000.00"
Oct-2023,"10,000.00",0.00,"-30,000.00","-2,500.00","-22,500.00"


## **Salaries**

In [ ]:
salaries_and_pension_and_statutory_contributions_df  = expenses.calculate_salaries_and_pension_and_statutory_contributions(
    new_disbursements_df=new_disbursements_df,
    parameters=parameters,
    months_to_forecast=MONTHS_TO_FORECAST,
    valuation_date=VALUATION_DATE,
)
salaries_and_pension_and_statutory_contributions_df

## **Provisions**


In [ ]:
provision_for_credit_loss_for_all_new_disbursements = (
    expenses.calculate_provision_for_credit_loss_for_all_new_disbursements(
        new_disbursements_df=new_disbursements_df, parameters=parameters
    )
)
provision_for_credit_loss_for_all_new_disbursements

### **Calculating Finance Costs**

In [ ]:
finance_costs = expenses.calculate_finance_costs(
    details_of_existing_long_term_borrowing = details_of_existing_long_term_borrowing,
    details_of_existing_short_term_borrowing = details_of_existing_short_term_borrowing,
    details_of_new_short_term_borrowing = details_of_new_short_term_borrowing,
    details_of_new_long_term_borrowing =details_of_new_long_term_borrowing,
    valuation_date= VALUATION_DATE,
    months_to_forecast= MONTHS_TO_FORECAST,
)

finance_costs.loc['total']

## **Income Statement**

In [ ]:
income_statement_index = pd.read_csv("./data_templates/income_statement_index.csv")
income_statement_index

In [ ]:
income_statement_df = income_statement.generate_income_statement_template(
    income_statement_index=income_statement_index,
    valuation_date=VALUATION_DATE,
    months_to_forecast=MONTHS_TO_FORECAST,
)
income_statement_df

### **Calculate Variable Expenses**

In [ ]:
variable_expenses = expenses.calculate_variable_expenses(
    variable_inputs_income_statement=variable_inputs_income_statement,
    parameters=parameters,
    valuation_date=VALUATION_DATE,
    months_to_forecast=MONTHS_TO_FORECAST,
)
variable_expenses

### **Calculate Change In Provision For Credit Loss**

In [ ]:
change_in_provision_for_credit_loss = (
    expenses.calculate_change_in_provision_for_credit_loss(
        provision_for_credit_loss=provision_for_credit_loss_for_all_new_disbursements['total'],
        provision_for_credit_loss_opening_balances=float(
            opening_balances["PROVISION_FOR_CREDIT_LOSS"].iat[0]
        ),
        valuation_date=VALUATION_DATE, 
        months_to_forecast=MONTHS_TO_FORECAST
    )
)

change_in_provision_for_credit_loss

### **Calculate Business Acquisition**

In [ ]:
business_acquisition = expenses.calculate_business_acqusition(
    business_acquisition_percent=parameters.loc['BUSINESS_ACQUISITION_PERCENT'],
    agent_contribution_percent=parameters.loc['AGENT_CONTRIBUTION_PERCENT'],
    consumer_ssb_disbursements=new_disbursements_df['consumer_ssb_disbursements'],
    consumer_pvt_disbursements=new_disbursements_df['consumer_pvt_disbursements'],
)
business_acquisition

### **Insert Revenue**

In [ ]:
income_statement_df = income_statement.insert_revenue(
    income_statement=income_statement_df,
    interest_income=total_interest_income,
    other_income=other_income_df['total'],
)

income_statement_df

### **Insert Static And Variable Inputs**

In [ ]:
income_statement_df = income_statement.insert_static_and_variable_inputs(
    income_statement=income_statement_df,
    static_inputs_income_statement=static_inputs_income_statement, variable_expenses=variable_expenses
)
income_statement_df

### **Insert Salaries and Pensions And Statutory Contributions**

In [ ]:
income_statement_df = income_statement.insert_salaries_and_pensions_and_statutory_contributions(
    income_statement=income_statement_df,
    salaries_and_pension_and_statutory_contributions_df = salaries_and_pension_and_statutory_contributions_df,
)

income_statement_df

### **Insert Depreciation**

In [ ]:
income_statement_df = income_statement.insert_depreciation(
    income_statement=income_statement_df, depreciation=depreciations_and_nbvs['dpns']['total'])

### **Insert Change in Provision For Credit Loss**

In [ ]:
income_statement_df = income_statement.insert_credit_loss_provision(
    income_statement=income_statement_df, 
    change_in_provisin_for_credit_loss=change_in_provision_for_credit_loss,
)

In [ ]:
provision_for_credit_loss  = provision_for_credit_loss_for_all_new_disbursements['total']
provision_for_credit_loss

### **Insert Business Acquisition**

In [ ]:
income_statement_df = income_statement.insert_business_acquisition(
    income_statement=income_statement_df, business_acquisition=business_acquisition)

### **Aggregations**

In [ ]:
income_statement_df = income_statement.aggregate_staff_costs(income_statement_df)
income_statement_df = income_statement.aggregate_travel_and_entertainment(income_statement_df)
income_statement_df = income_statement.aggregate_marketing_and_public_relations(income_statement_df)
income_statement_df = income_statement.aggregate_office_costs(income_statement_df)
income_statement_df = income_statement.aggregate_professional_fees(income_statement_df)
income_statement_df = income_statement.aggregate_communication_costs(income_statement_df)
income_statement_df = income_statement.aggregate_motor_vehicle_costs(income_statement_df)
income_statement_df = income_statement.aggregate_other_costs(income_statement_df)
income_statement_df = income_statement.aggregate_investment_income(income_statement_df)
income_statement_df = income_statement.aggregate_finance_costs(income_statement_df)

### **Calculating Total Expenses and EBIDTA**

In [ ]:
income_statement_df = income_statement.calculate_total_expenses(income_statement_df)
income_statement_df = income_statement.calculate_ebidta(income_statement_df)

### **Insert Finance Cost**

In [ ]:
income_statement_df.loc['Finance Costs'] = finance_costs.loc['total']

### **Aggregate Finance Cost**

In [ ]:
income_statement_df = income_statement.aggregate_finance_costs(income_statement_df)

### **Calculate Profit Before Tax**

In [ ]:
income_statement_df = income_statement.calculate_profit_before_tax(income_statement_df)

### **Calculate Tax**

In [ ]:
income_statement_df

In [ ]:
income_statement_df = income_statement.calculate_tax(income_statement=income_statement_df,tax_rate=parameters.loc['TAX_RATE'])
income_statement_df

## **Direct Cashflow**

In [ ]:
direct_cashflow_df = direct_cashflow.generate_direct_cashflow_template(
    valuation_date=VALUATION_DATE, months_to_forecast=MONTHS_TO_FORECAST
)

direct_cashflow_df

### **Insert Available Direct Cashflow Items**

#### **From Parameters**

In [ ]:
direct_cashflow_df.loc['Receipts From Trade Receivables'] = parameters.loc['RECEIPTS_FROM_TRADE_RECEIVABLES']
direct_cashflow_df.loc['Issue Of Shares'] = parameters.loc['ISSUE_OF_SHARES']
direct_cashflow_df.loc['Payments To Trade Payables'] = -parameters.loc['PAYMENTS_TO_TRADE_PAYABLES']
direct_cashflow_df.loc['Dividend Paid'] = -parameters.loc['DIVIDEND_PAID']

#### **From Calculations/Income Statement**

In [ ]:
direct_cashflow_df.loc['Interest Income'] = income_statement_df.loc['Interest Income']
direct_cashflow_df.loc['Other Income'] = income_statement_df.loc['Other Income']
direct_cashflow_df.loc['Interest Expense'] = -income_statement_df.loc['Finance Costs']
direct_cashflow_df.loc['Disbursements'] = -new_disbursements_df["total"].reindex(
    pd.PeriodIndex(new_disbursements_df["total"].index, freq="M").strftime("%b-%Y")
)

In [ ]:
direct_cashflow_df

### **Tax Paid**

In [ ]:
tax_schedule = direct_cashflow.generate_tax_schedule(
    taxation=income_statement_df.loc["Taxation"],
    opening_balance=opening_balances["DEFERED_TAXATION"].iat[0],
    valuation_date=VALUATION_DATE,
    months_to_forecast=MONTHS_TO_FORECAST,
)
tax_schedule

In [ ]:
direct_cashflow_df.loc['Tax Paid'] = tax_schedule.loc['Tax Paid']

In [ ]:
direct_cashflow_df

### **Calculate Operating Expenses**

In [ ]:
operating_expenses = direct_cashflow.calculate_operating_expenses(income_statement=income_statement_df)
operating_expenses

### **Insert Operating Expenses**

In [ ]:
direct_cashflow_df.loc['Operating Expenses']  = -operating_expenses
direct_cashflow_df


### **Calculate Capital Expenses**

In [ ]:
capital_expenses = direct_cashflow.calculate_capital_expenses(
    details_of_new_assets=details_of_new_assets,
    valuation_date=VALUATION_DATE,
    months_to_forecast=MONTHS_TO_FORECAST,
)


capital_expenses

### **Insert Capital Expenses**

In [ ]:
direct_cashflow_df.loc['Capital Expenses'] = -capital_expenses

### **Calculate Direct Cashflow Borrowing**

In [ ]:
long_and_short_term_borrowing_df = (
    direct_cashflow.calculate_long_and_short_term_borrowing_for_direct_cashflow(
        details_of_new_long_term_borrowing=details_of_new_long_term_borrowing,
        details_of_new_short_term_borrowing=details_of_new_short_term_borrowing,
        valuation_date=VALUATION_DATE,
        months_to_forecast=MONTHS_TO_FORECAST,
    )
)

long_and_short_term_borrowing_df

### **Insert Direct Cashflow Borrowing**

In [ ]:
direct_cashflow_df.loc['Short Term Borrowing'] = long_and_short_term_borrowing_df['short_term_borrowing']
direct_cashflow_df.loc['Long Term Borrowing'] = long_and_short_term_borrowing_df['long_term_borrowing']

In [ ]:
direct_cashflow_df

### **Calculate Capital Repayments**

In [ ]:
total_interest_income

In [ ]:
total_interest_income

In [ ]:
capital_repayment + total_interest_income

In [ ]:
capital_repayment  + interest_income_new_disbursements_df['total'].add(existing_loans_schedules['interest'].sum(), fill_value=0)

In [ ]:
capital_repayment = helper.add_series(
[existing_loans_schedules['capital_repayment'].sum()
    , capital_repayment_new_disbursements_df['total']]
)

capital_repayment

### **Insert Capital Repayment**

In [ ]:
direct_cashflow_df.loc['Capital Repayment'] = capital_repayment

### **Calculate Capital Repayments on Borrowings**


In [ ]:
capital_repayment_on_borrowings_df = direct_cashflow.calculate_capital_repayment_on_borrowings(
        details_of_existing_long_term_borrowing=details_of_existing_long_term_borrowing,
        details_of_existing_short_term_borrowing=details_of_existing_short_term_borrowing,
        details_of_new_short_term_borrowing=details_of_new_short_term_borrowing,
        details_of_new_long_term_borrowing=details_of_new_long_term_borrowing,
        valuation_date=VALUATION_DATE,
        months_to_forecast=MONTHS_TO_FORECAST,
    )

capital_repayment_on_borrowings_df

### **Insert Capital Repayment on New Borrowings**

In [ ]:
direct_cashflow_df.loc[
    "Capital Repayment On Borrowings"
] = -capital_repayment_on_borrowings_df['total']

### **Calculate Totals**

In [ ]:
direct_cashflow_df

In [ ]:
direct_cashflow_df.loc["Total Cash Inflows"] = direct_cashflow_df.iloc[
    direct_cashflow_df.index.get_loc("CASH INFLOWS")
    + 1 : direct_cashflow_df.index.get_loc("Total Cash Inflows")
].sum()


direct_cashflow_df.loc["Total Cash Outflows"] = direct_cashflow_df.iloc[
    direct_cashflow_df.index.get_loc("CASH OUTFLOWS")
    + 1 : direct_cashflow_df.index.get_loc("Total Cash Outflows")
].sum()

direct_cashflow_df.loc["Net Increase/Decrease In Cash"] = (
    direct_cashflow_df.loc["Total Cash Inflows"]
    + direct_cashflow_df.loc["Total Cash Outflows"]
)

### **Calculate Opening and Closing Balances**

In [ ]:
direct_cashflow_df

In [ ]:
direct_cashflow_df= direct_cashflow.calculate_opening_and_closing_balances_for_direct_cashflows(
direct_cashflow=direct_cashflow_df, 
cash_on_hand_opening_balance= opening_balances['CASH_ON_HAND'].iat[0]
)

direct_cashflow_df

### **Calculate IMTT (2% Tax)**

In [ ]:
income_statement_df.loc['2% Taxation']  = -direct_cashflow_df.loc['Total Cash Outflows']*IMTT

## **Calculate Profit and Loss for the Period**


In [ ]:
income_statement_df = income_statement.calculate_profit_or_loss_for_period(income_statement_df)
income_statement_df

## **Loan Book**

In [ ]:
loan_book_df = loan_book.generate_loan_book_template(
    valuation_date=VALUATION_DATE, months_to_forecast=MONTHS_TO_FORECAST
)
loan_book_df

### **Insert Loan Book Items**

In [ ]:
capital_repayment_existing_loans = existing_loans_schedules['capital_repayment'].sum()
capital_repayment_existing_loans

In [ ]:
total_capital_repayments = loan_book.aggregate_new_and_existing_loans_capital_repayments(
    capital_repayments_new_disbursements_df = capital_repayment_new_disbursements_df,
    capital_repayments_existing_loans = capital_repayment_existing_loans,
    valuation_date = VALUATION_DATE,
    months_to_forecast = MONTHS_TO_FORECAST,
)
total_capital_repayments

In [ ]:
loan_book_df = loan_book.insert_loan_book_items(
    loan_book=loan_book_df,
    opening_balance_on_loan_book=float(opening_balances["LOAN_BOOK"].iat[0]),
    total_interest_income=total_interest_income,
    total_capital_repayments=total_capital_repayments,
    disbursements=helper.change_period_index_to_strftime(new_disbursements_df["total"]),
)
loan_book_df

### **Calculate Opening and Closing Balances for Loan Book**

In [ ]:
loan_book_df = helper.calculate_opening_and_closing_balances(loan_book_df)
loan_book_df

## **Balance Sheet**


In [ ]:
balance_sheet_df = balance_sheet.generate_balance_sheet_template(
    valuation_date=VALUATION_DATE,
    months_to_forecast=MONTHS_TO_FORECAST,
)

balance_sheet_df

### **Insert Available Balance Sheet Items**

In [ ]:
depreciations_and_nbvs.keys()

In [ ]:
balance_sheet_df.loc["Property Plant And Equipment"] = depreciations_and_nbvs["nbvs"][
    "total"
]

balance_sheet_df.loc["Loan Book"] = loan_book_df.loc["Closing Balance"]

balance_sheet_df.loc["Cash On Hand"] = direct_cashflow_df.loc["Closing Balance"]

balance_sheet_df.loc[
    "Provisions"
] = provision_for_credit_loss_for_all_new_disbursements["total"]


balance_sheet_df.loc["Issued Share Capital"] = (
    parameters.loc["ISSUE_OF_SHARES"].cumsum()
    + opening_balances["ISSUED_SHARE_CAPITAL"].iat[0]
)

balance_sheet_df.loc["Provision For Taxation"] = helper.change_period_index_to_strftime(
    parameters.loc["PROVISION_FOR_TAX"]
)

balance_sheet_df.loc["Other Payables"] = helper.change_period_index_to_strftime(
    parameters.loc["OTHER_PAYABLES"]
)
balance_sheet_df.loc["Intercompany Loans"] = helper.change_period_index_to_strftime(
    parameters.loc["INTERCOMPANY_LOANS"]
)

balance_sheet_df

### **Generate Borrowinds Schedules**

In [ ]:
short_term_loans_schedules = balance_sheet.calculate_short_term_loans_schedules(
    long_and_short_term_borrowing_df=long_and_short_term_borrowing_df,
    capital_repayment_on_borrowings_df=capital_repayment_on_borrowings_df,
    opening_balances=opening_balances,
    valuation_date=VALUATION_DATE, 
    months_to_forecast=MONTHS_TO_FORECAST
)

short_term_loans_schedules

In [ ]:
long_term_loans_schedules = balance_sheet.calculate_long_term_loans_schedules(
    long_and_short_term_borrowing_df=long_and_short_term_borrowing_df,
    capital_repayment_on_borrowings_df=capital_repayment_on_borrowings_df,
    opening_balances=opening_balances,
    valuation_date=VALUATION_DATE, 
    months_to_forecast=MONTHS_TO_FORECAST
)

long_term_loans_schedules

### **Insert Borrowings and Loans**


In [ ]:
balance_sheet_df.loc["Loans"] = long_term_loans_schedules.loc['Closing Balance']
balance_sheet_df.loc["Borrowings"] = short_term_loans_schedules.loc['Closing Balance']

### **Generate Trade Receivables and Payables Schedule**

In [ ]:
trade_receivables_schedule = expenses.generate_trade_receivables_schedule(
    opening_trade_receivables=opening_balances['TRADE_RECEIVABLES'].iat[0], 
    receipts_from_trade_receivables=parameters.loc["RECEIPTS_FROM_TRADE_RECEIVABLES"], 
    new_trade_receivables=parameters.loc['NEW_TRADE_RECEIVABLES'], 
    months_to_forecast=MONTHS_TO_FORECAST, 
    valuation_date=VALUATION_DATE
)

trade_receivables_schedule


In [ ]:
balance_sheet_df.loc['Deferred Taxation'] = tax_schedule.loc['Closing Balance']


In [ ]:
trade_payables_schedule = balance_sheet.generate_trade_payables_schedule(
    opening_trade_payables=opening_balances["TRADE_PAYABLES"].iat[0],
    payments_to_trade_payables=parameters.loc["PAYMENTS_TO_TRADE_PAYABLES"],
    new_trade_payables=parameters.loc["NEW_TRADE_PAYABLES"],
    months_to_forecast=MONTHS_TO_FORECAST,
    valuation_date=VALUATION_DATE,
)

trade_payables_schedule

In [ ]:
trade_receivables_schedule

In [ ]:
parameters.loc[parameters.index.str.contains(pat = 'TRADE', na = False)]

In [ ]:
balance_sheet_df.loc['Trade Payables'] = trade_payables_schedule.loc['Closing Balance']
balance_sheet_df.loc["Trade Receivables"] = trade_receivables_schedule.loc[
    "Closing Balance"
]
balance_sheet_df

### **Capital and Reserves**

In [ ]:
balance_sheet_df.loc["Issued Share Capital"] = (
    parameters.loc["ISSUE_OF_SHARES"].cumsum()
    + opening_balances["ISSUED_SHARE_CAPITAL"].iat[0]
)
balance_sheet_df.loc["Share Premium"] = opening_balances["SHARE_PREMIUM"].iat[0]
balance_sheet_df.loc["Other Components Of Equity"] = opening_balances[
    "OTHER_COMPONENTS_OF_EQUITY"
].iat[0]
balance_sheet_df.loc["Treasury Shares"] = opening_balances["TREASURY_SHARES"].iat[0]
balance_sheet_df.loc["Retained Earnings"] = (
    income_statement_df.loc["PROFIT/(LOSS) FOR PERIOD"]
    - parameters.loc["DIVIDEND_PAID"].reindex(
        parameters.loc["DIVIDEND_PAID"].index.strftime("%b-%Y")
    )
).cumsum()
balance_sheet_df.loc["Capital And Reserves"] = balance_sheet_df.loc[
    "Issued Share Capital":"Retained Earnings"
].sum()

In [ ]:
balance_sheet_df

In [ ]:
balance_sheet_df = balance_sheet.calculate_other_assets(balance_sheet_df=balance_sheet_df, parameters=parameters, opening_balances=opening_balances)
balance_sheet_df

### **Calculate Totals**


In [ ]:
balance_sheet_df = balance_sheet.sum_financial_statements_totals(balance_sheet_df)
balance_sheet_df = balance_sheet.calculate_final_balances(balance_sheet_df=balance_sheet_df)
balance_sheet_df

## **Indirect Cashflow**

In [ ]:
statement_of_cashflow_df = (
    statement_of_cashflows.generate_statement_of_cashflow_template(
        VALUATION_DATE, MONTHS_TO_FORECAST
    )
)

In [ ]:
change_in_trade_payables = trade_payables_schedule.loc['Closing Balance'] - trade_payables_schedule.loc['Opening Balance']
change_in_trade_receivables = trade_receivables_schedule.loc['Opening Balance'] - trade_receivables_schedule.loc['Closing Balance']
change_in_loan_book_principle = loan_book_df.loc['Opening Balance'] - loan_book_df.loc['Closing Balance'] 


### **Inserting Readily Available Figures**

In [ ]:
statement_of_cashflow_df.loc['Profit/(loss) per I/S'] = income_statement_df.loc['PROFIT/(LOSS) FOR PERIOD']
statement_of_cashflow_df.loc['Depreciation'] = income_statement_df.loc['Depreciation']
statement_of_cashflow_df.loc['(Increase)/Decrease in Receivables'] = change_in_trade_receivables
statement_of_cashflow_df.loc['Increase/(Decrease) in Payables'] = change_in_trade_payables
statement_of_cashflow_df.loc['(Increase)/Decrease in Loan Book (Principle)'] = change_in_loan_book_principle
statement_of_cashflow_df.loc['Dividend Paid'] = helper.change_period_index_to_strftime(parameters.loc["DIVIDEND_PAID"])
statement_of_cashflow_df.loc['Interest Paid'] = finance_costs.loc['total']
statement_of_cashflow_df.loc['Tax Paid'] = tax_schedule.loc['Tax Paid']
statement_of_cashflow_df.loc['Repayment of Borrowings'] = capital_repayment_on_borrowings_df['total']
statement_of_cashflow_df.loc['Purchase of Fixed Assets'] = capital_expenses
statement_of_cashflow_df.loc["Increase/(Decrease) in Short Term Borrowings"] = (
    short_term_loans_schedules.loc["Closing Balance"]
    - short_term_loans_schedules.loc["Opening Balance"]
)

statement_of_cashflow_df.loc["Increase/(Decrease) in Long Term Borrowings"] = (
    long_term_loans_schedules.loc["Closing Balance"]
    - long_term_loans_schedules.loc["Opening Balance"]
)
statement_of_cashflow_df

In [ ]:
statement_of_cashflow_df = balance_sheet.sum_financial_statements_totals(statement_of_cashflow_df)
statement_of_cashflow_df

In [ ]:
statement_of_cashflow_df.fillna(0, inplace=True)

In [ ]:
statement_of_cashflow_df

### **Calculating Totals**

In [ ]:
statement_of_cashflow_df.loc['Net Increase/(Decrease) in Cash'] = (
    statement_of_cashflow_df.loc["Net Cash Flow From Operations"]
    + statement_of_cashflow_df.loc["Cash Flow From Investing Activities"]
    + statement_of_cashflow_df.loc["Cash Flow From Financing Activities"]
)

In [ ]:
statement_of_cashflow_df

In [ ]:
statement_of_cashflow_df = (
    statement_of_cashflows.calculate_cash_at_end_and_beginning_of_period(
        statement_of_cashflow_df=statement_of_cashflow_df,
        opening_balances=opening_balances,
    )
)
statement_of_cashflow_df